In [1]:
from datetime import datetime
import json
from typing import List

from arcgis.features import GeoAccessor
from arcgis.geometry import Point
from arcgis.gis import GIS
import h3
import pandas as pd
from requests import Session

In [2]:
yelp_client_id = 'ZAyA5LhP0CVLQ3sJwFX2SA'
yelp_api_key = 'SDXO_3tR2DH1XbHCDKjKZ11q_TixjKnKicvLMKSa5CuPE3guusBwatV50ETtmxLrJRBk6g2EkUQTI1FY4Jni_7enVDUF2g19IrcAzE3xaqOLyB0sUFFIMtT4NrX3WXYx'

yelp_business_search_url = 'https://api.yelp.com/v3/businesses/search'
h3_res = 9

In [3]:
def add_h3_index(yelp_item: List[dict], h3_resolution: int = 9) -> List[dict]:
    
    # get the h3 index
    h3_idx = h3.latlng_to_cell(yelp_item.get('coordinates').get('latitude'), yelp_item.get('coordinates').get('longitude'), h3_resolution)
    
    # add the h3 index
    yelp_item[f'h3_{h3_resolution:02d}'] = h3_idx
    
    return yelp_item


def add_geometry(yelp_item: List[dict], format: str = 'esri') -> List[dict]:
    
    # get the coordinates
    coord_x = yelp_item.get('coordinates').get('longitude')
    coord_y = yelp_item.get('coordinates').get('latitude')
    
    # if esri, create ArcGIS Point geometry
    if format == 'arcgis':
        
        pt = Point({'x': coord_x, 'y': coord_y, 'spatialReference' : {'wkid' : 4326}})
    
    # if geojson, do that
    elif format == 'geojson':
        
        pt = {
          "type": "Point",
          "coordinates": [coord_x, coord_y]
        }

    # add to the Yelp item
    yelp_item['geometry'] = pt
    
    return yelp_item


def prep_yelp_item(yelp_item: dict) -> dict:

    # get a string formatted address for display
    loc = yelp_item.get('location')
    if loc is not None:
        yelp_item['address'] = ', '.join(loc.get('display_address'))

    # get string formatted categories for display
    cats = yelp_item.get('categories')
    if cats is not None:
        cats = [cat.get('title') for cat in cats]
        cats.sort()
        yelp_item['categories'] = ', '.join(cats)

    # prune the dictionary
    keep_keys = ['id', 'name', 'image_url', 'url', 'review_count', 'categories', 'rating', 'price', 'address']
    keep_keys = [k for k in yelp_lst[0].keys() if k in keep_keys or k.startswith('h3_') or k == 'geometry']
    yelp_item = dict((k, yelp_item.get(k)) for k in keep_keys)

    return yelp_item

In [4]:
headers = {
    'accept': 'application/json',
    'Authorization': f'Bearer {yelp_api_key}'
}

In [5]:
loc_x, loc_y = -122.91479355798339, 47.042924037902935  # house in Olympia
# loc_x, loc_y = -105.4468809729821, 20.488120802895565  # where we stayed in Yelapa

In [6]:
yelp = Session()
yelp.headers = headers

In [7]:
params = {
    'limit': 50,
    'sort_by': 'best_match',
    'radius': 1600,  # straight-line distance in meters
    'longitude': loc_x,
    'latitude': loc_y,
    'term': 'restaurants'
}

In [8]:
res = yelp.get(yelp_business_search_url, params=params)

if res.status_code == 200:
    
    # get the list of businesses from the response
    yelp_lst = res.json().get('businesses')
    
    # add the level h3 level 7 through 11 indices
    for h3_lvl in range(7, 12):
        yelp_lst = [add_h3_index(yelp_itm, h3_lvl) for yelp_itm in yelp_lst]
        
    # add ArcGIS Python API point geometry
    yelp_lst = [add_geometry(yelp_itm, format="geojson") for yelp_itm in yelp_lst]
    
    # format output to be more usable with ArcGIS
    yelp_lst = [prep_yelp_item(yelp_itm) for yelp_itm in yelp_lst]

len(yelp_lst)

50

In [9]:
yelp_item = yelp_lst[0]

yelp_item

{'id': 'oN0jxgY7aBj_tYLgTIKcNw',
 'name': 'Uptown Grill',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/SRK7mc61iD5hzoqa3GsjJg/o.jpg',
 'url': 'https://www.yelp.com/biz/uptown-grill-olympia?adjust_creative=ZAyA5LhP0CVLQ3sJwFX2SA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ZAyA5LhP0CVLQ3sJwFX2SA',
 'review_count': 344,
 'categories': 'American, Burgers, Cocktail Bars',
 'rating': 4.5,
 'price': '$$',
 'h3_07': '8728d5910ffffff',
 'h3_08': '8828d59103fffff',
 'h3_09': '8928d591037ffff',
 'h3_10': '8a28d591034ffff',
 'h3_11': '8b28d5910348fff',
 'geometry': {'type': 'Point', 'coordinates': [-122.901333, 47.043801]},
 'address': '514 Capitol Way S, Olympia, WA 98501'}

In [94]:
df = pd.DataFrame.from_records(yelp_lst)
df.spatial.set_geometry('geometry', sr=4326)

df

,id,name,image_url,url,review_count,categories,rating,price,h3_07,h3_08,h3_09,h3_10,h3_11,geometry,address
0,oN0jxgY7aBj_tYLgTIKcNw,Uptown Grill,https://s3-media3.fl.yelpcdn.com/bphoto/SRK7mc...,https://www.yelp.com/biz/uptown-grill-olympia?...,344,"American, Burgers, Cocktail Bars",4.5,$$,8728d5910ffffff,8828d59103fffff,8928d591037ffff,8a28d591034ffff,8b28d5910348fff,"{""x"": -122.901333, ""y"": 47.043801, ""spatialRef...","514 Capitol Way S, Olympia, WA 98501"
1,vmFXQJ7MsCnEotwR8qY8PA,Chicory,https://s3-media2.fl.yelpcdn.com/bphoto/M7m8c9...,https://www.yelp.com/biz/chicory-olympia?adjus...,107,"Bars, New American, Seafood",4.5,$$,8728d5911ffffff,8828d5911dfffff,8928d5911cbffff,8a28d5911c9ffff,8b28d5911c98fff,"{""x"": -122.903090491891, ""y"": 47.0451905457797...","111 Columbia St NW, Olympia, WA, Olympia, WA 9..."
2,nt17g2s7dOjOIKJLi7AlYQ,Row,https://s3-media1.fl.yelpcdn.com/bphoto/aSHxBY...,https://www.yelp.com/biz/row-olympia?adjust_cr...,159,New American,4.0,$$,8728d5911ffffff,8828d5911dfffff,8928d591153ffff,8a28d5911527fff,8b28d5911526fff,"{""x"": -122.9038764, ""y"": 47.0457683, ""spatialR...","208 State Ave NW, Olympia, WA 98501"
3,5diQLeEnRaEiI9gYMC-UXg,1889 Prime Steak House,https://s3-media4.fl.yelpcdn.com/bphoto/POG3Cn...,https://www.yelp.com/biz/1889-prime-steak-hous...,35,Steakhouses,4.0,None,8728d5911ffffff,8828d5911dfffff,8928d5911cbffff,8a28d5911caffff,8b28d5911ca8fff,"{""x"": -122.9056458, ""y"": 47.04390685157421, ""s...","406 5th Ave SW, Olympia, WA 98501"
4,8LdL1xJwOE7TXHTD-L0vFg,Olympia Oyster House,https://s3-media2.fl.yelpcdn.com/bphoto/_ToEC3...,https://www.yelp.com/biz/olympia-oyster-house-...,471,Seafood,4.0,$$,8728d5911ffffff,8828d5911dfffff,8928d5911cbffff,8a28d5911c8ffff,8b28d5911c8bfff,"{""x"": -122.90519, ""y"": 47.04512, ""spatialRefer...","320 4th Ave W, Olympia, WA 98501"
5,RucjyhLr6QiHdlQH1eKaoA,Cascadia Grill,https://s3-media2.fl.yelpcdn.com/bphoto/cZjjKc...,https://www.yelp.com/biz/cascadia-grill-olympi...,554,"Bars, Brasseries, Steakhouses",4.5,$$,8728d5911ffffff,8828d5911dfffff,8928d5911cbffff,8a28d5911c9ffff,8b28d5911c9cfff,"{""x"": -122.90313455582186, ""y"": 47.04489344348...","200 4th Ave W, Olympia, WA 98501"
6,S_pz_5iAspiZ89C0FpmvqQ,Cynara Restaurant & Lounge,https://s3-media4.fl.yelpcdn.com/bphoto/05U3BZ...,https://www.yelp.com/biz/cynara-restaurant-and...,118,New American,4.0,None,8728d5911ffffff,8828d59115fffff,8928d591143ffff,8a28d5911437fff,8b28d5911434fff,"{""x"": -122.9028812, ""y"": 47.0486969, ""spatialR...","500 Columbia St NW, Olympia, WA 98501"
7,HeieXG-dNUTb9aRNP-tPxw,Octapas Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/fAHFay...,https://www.yelp.com/biz/octapas-cafe-olympia?...,201,"Breakfast & Brunch, Cocktail Bars, Tapas/Small...",4.5,$$,8728d5911ffffff,8828d5911dfffff,8928d5911cbffff,8a28d5911ca7fff,8b28d5911ca6fff,"{""x"": -122.9036089, ""y"": 47.042810716779464, ""...","610 Water St SW, Olympia, WA 98501"
8,czlRjPU2z7QVeX5Yp4rQiw,The Owl's Nest coffee + pastries,https://s3-media2.fl.yelpcdn.com/bphoto/kFDGeR...,https://www.yelp.com/biz/the-owls-nest-coffee-...,49,"Bakeries, Breakfast & Brunch, Coffee & Tea",4.5,None,8728d5911ffffff,8828d59103fffff,8928d591027ffff,8a28d591026ffff,8b28d5910268fff,"{""x"": -122.90154, ""y"": 47.04549, ""spatialRefer...","116 Capitol Way N, Olympia, WA 98501"
9,hkTnpSYCACbUzgya4LiwZg,Dockside Bistro & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/KXIaEw...,https://www.yelp.com/biz/dockside-bistro-and-w...,300,"New American, Seafood, Wine Bars",4.0,$$$,8728d5911ffffff,8828d59115fffff,8928d591153ffff,8a28d591151ffff,8b28d5911519fff,"{""x"": -122.90398, ""y"": 47.048308, ""spatialRefe...","501 Columbia St NW, Olympia, WA 98501"


In [95]:
df.spatial.validate()

True

In [97]:
gis = GIS(username='knu2xs')

gis

Enter password:  ········


GIS @ https://knu2xs.maps.arcgis.com version:2023.3

In [98]:
df.spatial.to_featurelayer('yelp_test_items', gis=gis)

<Item title:"yelp_test_items" type:Feature Layer Collection owner:knu2xs>